<a href="https://colab.research.google.com/github/TonyQ2k3/pytorch-training/blob/main/notebooks/pytorch_day7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch Day 7 - Computer Vision with Pytorch
---

## Convolutional Neural Network

A type of deep learning model commonly used for analysing visual data.

Key components:
+ Apply convolutional operations to detect edges and shapes.
+ Hidden layer activation function: ReLU

In [2]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt